# 自分で作成をした特徴量がメイン

# 参考にしたurl
【随時更新】Kaggleテーブルデータコンペできっと役立つTipsまとめ  
https://naotaka1128.hatenadiary.jp/entry/kaggle-compe-tips  
最近のKaggleに学ぶテーブルデータの特徴量エンジニアリング
https://www.slideshare.net/mlm_kansai/kaggle-138546659

### ライブラリのimport

In [ ]:
import gc
import os
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from datetime import datetime,timedelta
import pickle
import os, sys, gc, time, warnings, pickle, psutil, random
from scipy import stats
from scipy.special import inv_boxcox
from typing import Tuple
from datetime import timedelta
import psutil

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

### メモリの使用量の制限

In [ ]:
#これは使い回しができるはず！
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df
#実際に使う場合はこれを書く。
#df=df.pipe(reduce_mem_usage())

### メモリの使用量の確認

In [ ]:
import psutil
mem = psutil.virtual_memory() 
print(mem.percent)

### なにがメモリを食っているかを確認が出来る関数

In [ ]:
def print_varsize():
    import types
    print("{}{: >15}{}{: >10}{}".format('|','Variable Name','|','  Size','|'))
    print(" -------------------------- ")
    for k, v in globals().items():
        if hasattr(v, 'size') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(v.size),'|'))
        elif hasattr(v, '__len__') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(len(v)),'|'))
print_varsize()

### 複数のデータを取り込む

In [ ]:
#データの読み込みを行う→これは既にevaluationに変わっている
def get_data_by_store():
    df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                    pd.read_pickle('itou_lag.pkl'),
                    pd.read_pickle('itou_laglog.pkl')],
                    axis=1)
    df = df[df['store_id']==store]
    df2 = pd.read_pickle('mean_encoding.pkl')[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle('lags_df_28.pkl').iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 
    
    df = pd.concat([df, df3], axis=1)
    del df3 
    
    
    #実際にlightgbmを動かすときにはいらないので注意が必要
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    #ここの設定をいくつにするか問題はある！
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df

### 時系列データの処理

数字データを時系列データに変換する

In [ ]:
#日付に関する特徴量を作成するには年の表記を元に戻さないといけない
tm_y=[0,1,2,3,4,5]
year=[2011,2012,2013,2014,2015,2016]
#年数を普通の西暦に直す
ddf['year']=ddf['tm_y'].replace(tm_y,year)
#年の表記の仕方も変える
ddf['year']=ddf['year'].astype(str)
#月の表記の仕方を変える。例えば6→06に！
ddf['month']=[n.zfill(2) for n in ddf['tm_m'].astype(str)]
#日にちの表記の仕方を変える
ddf['date']=[n.zfill(2) for n in ddf['tm_d'].astype(str)]
#数字から時系列データに戻す
ddf['datetime'] = pd.to_datetime(ddf['year']+ddf['month']+ddf['date'], format='%Y%m%d')

日付のデータから基本的な時系列データの作成

In [ ]:
#日付に関するデータ作成
date_features = {
            "wday": "weekday",
            "week": "weekofyear",
            "month": "month",
            "quarter": "quarter",
            "year": "year",
            "mday": "day",
            #多分weekdayと値は同じになる予想
            "dayofweek":"dayofweek",
            "is_year_end":"is_year_end",
            "is_year_start":"is_year_start",
            "is_quarter_end":"is_quarter_end",
            "is_quarter_start":"is_quarter_start",
            "is_month_end":"is_month_end",
            "is_month_start":"is_month_start",
        }
#date_featuresを事前に設定をしておいて、特徴的な時系列データを特徴量に加える
for date_feat_name, date_feat_func in date_features.items():
        #date_featuresにも元データにもあるものは'int16'にしてデータのメモリを抑える
    if date_feat_name in ddf.columns:
        ddf[date_feat_name] = ddf[date_feat_name].astype("int16")
        #date_featuresの中でないやつは新しく作成をする。
        #→ないのはweek,quarter,mday,daofweek以降のやつ
    else:
        ddf[date_feat_name] = getattr(ddf["datetime"].dt, date_feat_func).astype("int16")

季節に関する特徴量

In [ ]:
#季節に関する特徴量！'tm_m'には月ごとの数値が入っている。
ddf['season']=0
ddf['season']=[1 if 3<=n<=5 else 2 if 6<=n <=8 else 3 if 9<=n <=11 else 4 for n in ddf['tm_m']]

### 汎用的な特徴量作成

0かどうかをcountする

In [ ]:
#np.whereはif文とかなり特徴が似ている
ddf["release_0"]=np.where(ddf["release"]==0,1,0) 
#出来ればお店の種類ごとにやりたいと想ったが既にお店ごとに分けているやつ
ddf["release_0_count"]=ddf.groupby(["id"])["release_0"].transform('count')

ある値かどうかを内包表記を使って変数の作成

In [ ]:
#新発売ならば1を立てる！→直近1週間とかも出来そう！
ddf['weekly_release']=[1 if 1<=n<=7 else 0 for n in ddf['release']]
ddf['monthly_release']=[1 if 1<=n<=30 else 0 for n in ddf['release']]
ddf['after_release']=[1 if 1<=n<=30 else 0 for n in ddf['release']]

カテゴリー変数に対してダミー変数を作成する

In [ ]:
#get_dummiesでone-hot-encoderが出来た！
#prefixで列名の指定が可能である。
tmp1 = pd.get_dummies(ddf['event_name_1'], prefix='event')
tmp2 = pd.get_dummies(ddf['event_name_2'], prefix='event')

小数点以下だけ取り出した変数の作成   
→4ドルと3.98ドルは人間心理的にぜんぜん違うよ

In [ ]:
#整数のみの特徴量を作成する
ddf['sell_price_int']=ddf['sell_price'].astype('int16')
# 商品価格について小数点以下のみを残す。購買意欲に関わるはず。
ddf['sell_price_float']=ddf['sell_price']-ddf['sell_price_int']
del ddf['sell_price_int']

累積和を使いたいとき  
→今回で言うとsalesの平均値とのブレを一定期間で測ってみるみたいなこと

In [ ]:
ddf['difference_price1_cumsum']= ddf.groupby('id')['price_change_cat'].cumsum()

targetencoding  
→普通にleakageを起こすので注意をする

In [ ]:
#'id'以外のカテゴリ間の平均値の算出
mean_encoding_combination = [
    ['dept_id'], 
    ['item_id'], 
    ['dept_id','tm_y'], 
    ['item_id','tm_y'],
    ['dept_id', 'tm_m'],
    ['dept_id', 'tm_m', 'tm_dw'],
    ['dept_id', 'tm_dw' ,'snap_CA'],
    ['dept_id', 'tm_dw' ,'snap_TX'],
    ['dept_id', 'tm_dw' ,'snap_WI']
]
#実際に作成をしていく
for col in mean_encoding_combination:
    print(col, 'encoding')
    colnm1 = '_'.join(col)+'_mean_cat'
    colnm2 = '_'.join(col)+'_std_cat'
    ddf[colnm1] = ddf.groupby(col)['sell_price'].transform(lambda x:x.mean())
    ddf[colnm2] = ddf.groupby(col)['sell_price'].transform(lambda x:x.std())

lag特徴量の作成

In [ ]:
lag_list=[7,28]
for lag in lag_list:
    #7日おきにlag変数を作成が出来る
    ddf[f'sales_lag_{lag}'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.shift(lag)).astype(np.float16)

groupbyによる基本特徴量の作成

歪度(Skewness)と尖度(Kurtosis)  
歪度(Skewness) : 分布の非対称性を示す指標。  
値が正であると分布が右に伸びている。逆に負であると左に伸びている。

尖度(Kurtosis) : 正規分布を基準とした分布の鋭さの指標。  
正規分布の尖度を0と定義する場合と3と定義する場合がある。基準である正規分布の尖度よりも大きければ分布は尖っており、小さければ緩やかな分布である。

In [ ]:
ddf['sales_mean'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.mean()).astype(np.float16)
ddf['sales_std'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.std()).astype(np.float16)
ddf['sales_max'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.max()).astype(np.float16)
df['sales_min'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.min()).astype(np.float16)
ddf['sales_kurt'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.kurt()).astype(np.float16)
ddf['sales_skew'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.skew()).astype(np.float16)

移動平均を用いた特徴量の作成

In [ ]:
#lag28を基準にrollingさせて,基本統計量を算出
d_window=[28,56]
for d in d_window:
    print(d)
    ddf[f'log28_mean_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).mean()).astype(np.float16)
    ddf[f'log28_std_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).std()).astype(np.float16)
    #ここから下はやる意味あるかは微妙
    ddf[f'log28_max_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).max()).astype(np.float16)
    ddf[f'log28_min_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).min()).astype(np.float16)
    ddf[f'log28_kurt_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).kurt()).astype(np.float16)
    ddf[f'log28_skew_{d}'] = ddf.groupby(['id'])['logs_28'].transform(lambda x: x.rolling(d).skew()).astype(np.float16)

対数変換  
→対数変換をする意味はこれhttps://atarimae.biz/archives/13161

In [ ]:
#目的は外れ値の影響を受けづらくして、時系列の変化をとらえやすくするため。
def log_transformation(x: pd.Series) -> pd.Series:
    # Function np.log1p = log(x + 1)
    return np.log1p(x)
#直近2週間のlagデータって使えそうだが、まだなかったので作っておく
lag_list=[7,28]
for i in lag_list:
    ddf[f'sales_lag_{i}'] = ddf.groupby(['id'])['sales'].transform(lambda x: x.shift(i)).astype(np.float16) 

### データの保存

In [ ]:
#ここを新規に作成をした列のみにする
ddf_time=ddf.iloc[:,30:].pipe(reduce_mem_usage)
#pklファイルとして保存をしておく
ddf_time.to_pickle('itou_time.pkl')

### メモリの使用量の管理

In [ ]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print(mem.percent)
#なにがメモリを食っているかを確認することが出来る
def print_varsize():
    import types
    print("{}{: >15}{}{: >10}{}".format('|','Variable Name','|','  Size','|'))
    print(" -------------------------- ")
    for k, v in globals().items():
        if hasattr(v, 'size') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(v.size),'|'))
        elif hasattr(v, '__len__') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(len(v)),'|'))
print_varsize()